# 複数のトランザクションを同時に送信
## 概要
- 秘密鍵を有しているウォレットから複数の送金トランザクションを同時に送信

## 注意
- 秘密鍵を公開してしまうとウォレットからモザイクが盗まれる可能性があるので、間違って公開してしまわないようにしましょう。

In [7]:
# 送信先アドレス、送金量、メッセージ（送金量は可分性が6なので1000000倍した数値を指定）
TO_LIST = [
    ["TBMAGI3FR6ZFDXB5CHYTCE5EBKBSQP2CZPS7CGA", 1 * 1000000, "Hi Bob."],
    ["TAJEWDXCCMQLWTKVWDAX2EW7KS3L2NT6BOOIDSQ", 2 * 1000000, "Hi Alice."],
    ["TBQFD5L6OPJHHNJWCANKH6GG4GF3RRZOXNQ4KUA", 3 * 1000000, "こんにちは、太郎さん"]
]

In [8]:
# 送信元
FROM_PRIVATEKEY = "270A5053FEDDAD47B05216D270A2F9E6DDCE74A8A16E8E3410022BDE011A3***"

# 最大手数料 (この例だと0.1枚)
MAX_FEE = 1 * 100000

# Symbol誕生のUTC秒 (メインネットは1615853185)
BIRTHTIME = 1616694977

# トランザクションの有効期限(単位はhour。この例だと2時間)
EXP_TIME = 2

# XYMのモザイクID (メインネットは0x6BED913FA20223F8)
MOSAIC_ID = 0x091F837E059AE13C

# トランザクションの送信先ノード
NODEURL = "http://sym-test-01.opening-line.jp:3000"

In [9]:
import datetime
import urllib.request
import json
from binascii import unhexlify, hexlify
from symbolchain.core.CryptoTypes import PrivateKey, Hash256
from symbolchain.core.sym.KeyPair import KeyPair
from symbolchain.core.facade.SymFacade import SymFacade
from symbolchain.core.sym.MerkleHashBuilder import MerkleHashBuilder
import sha3

facade = SymFacade('public_test')
keypair = KeyPair(PrivateKey(unhexlify(FROM_PRIVATEKEY)))

In [10]:
# トランザクションのリスト作成
tx_lst = []
for item in TO_LIST:
    tx = facade.transaction_factory.create_embedded({
        'type': 'transfer',
        'signer_public_key': keypair.public_key,
        'recipient_address': SymFacade.Address(item[0]),
        'mosaics': [(MOSAIC_ID, item[1])],
        'message': item[2]
    })
    tx_lst.append(tx)

# マークルハッシュの作成
hash_builder = MerkleHashBuilder()
for tx in tx_lst:
    hash_builder.update(Hash256(sha3.sha3_256(tx.serialize()).digest()))
merkle_hash = hash_builder.final()

# アグリゲートトランザクションの有効期限
deadline = (int((datetime.datetime.today() + datetime.timedelta(hours=EXP_TIME)).timestamp()) - BIRTHTIME) * 1000

# アグリゲートコンプリートの作成
aggregate = facade.transaction_factory.create({
    'type': 'aggregateComplete',
    'signer_public_key': keypair.public_key,
    'fee': MAX_FEE,
    'deadline': deadline,
    'transactions_hash': merkle_hash,
    'transactions': tx_lst
})

# 自己署名
signature = facade.sign_transaction(keypair, aggregate)
aggregate.signature = signature.bytes

# アグリゲートトランザクションのハッシュ値
tx_hash = facade.hash_transaction(aggregate)

# ノードへアナウンス
payload = {"payload": hexlify(aggregate.serialize()).decode('utf8').upper()}
req = urllib.request.Request(NODEURL + "/transactions",
                             json.dumps(payload).encode(),
                             {'Content-type': 'application/json'},
                             method='PUT')
with urllib.request.urlopen(req) as res:
    print("tx hash:" + str(tx_hash))
    print("status code:" + str(res.getcode()))
    print('http://explorer.testnet.symboldev.network/transactions/' + str(tx_hash))

tx hash:CD321C0840D9DFCEE419A2D3C2281BFC818963F3C836F51CB47C0237E8E85DC0
status code:202
http://explorer.testnet.symboldev.network/transactions/CD321C0840D9DFCEE419A2D3C2281BFC818963F3C836F51CB47C0237E8E85DC0


In [11]:
# 結果確認
req = urllib.request.Request(NODEURL + "/transactionStatus/" + str(tx_hash))
with urllib.request.urlopen(req) as res:
    data = json.load(res)
    print(json.dumps(data, indent=2))

{
  "group": "confirmed",
  "code": "Success",
  "hash": "CD321C0840D9DFCEE419A2D3C2281BFC818963F3C836F51CB47C0237E8E85DC0",
  "deadline": "10698378000",
  "height": "275528"
}


In [22]:
# メッセージ確認対象のアドレス
TO_ADDRESS = "TBMAGI3FR6ZFDXB5CHYTCE5EBKBSQP2CZPS7CGA"
# トランザクションのタイプ(0x4141 (16705 decimal) - AggregateCompleteTransaction.)
tx_type = 0x4141

req = urllib.request.Request(NODEURL + '/accounts/' + TO_ADDRESS)
with urllib.request.urlopen(req) as res:
    accountInfo = json.load(res)
ADDRESS48 = accountInfo['account']['address']

url = NODEURL + '/transactions/confirmed'
params = {
    'address': TO_ADDRESS,
    'type': tx_type,
    'order': 'desc',
}

req = urllib.request.Request('{}?{}'.format(url, urllib.parse.urlencode(params)))
with urllib.request.urlopen(req) as res:
    data = json.load(res)

for d in data['data']:
    print(d['id'])

60FFF136A36C2904DF689E49
60FFF089A36C2904DF689E35


In [30]:
# トランザクションのタイプ(0x4154 (16724 decimal) - TransferTransaction.)
tx_type = 0x4154
# XYMのモザイクID(メインネットは6BED913FA20223F8)
XYM_ID = '091F837E059AE13C'

# 詳細を取得するアグリゲートトランザクションのid
tx_ids = ['60FFF136A36C2904DF689E49']

url = NODEURL + '/transactions/confirmed'
params = {
    'transactionIds': tx_ids
}
req = urllib.request.Request(url,
                             json.dumps(params).encode(),
                             {'Content-type': 'application/json'},
                             method='POST')
with urllib.request.urlopen(req) as res:
    data = json.load(res)

for d in data:
    for tx in d['transaction']['transactions']:
        # 自分宛の送金トランザクションのみ抽出
        if tx['transaction']['type'] == tx_type and tx['transaction']['recipientAddress'] == ADDRESS48:
            message = ""
            if "message" in tx['transaction']:
                message = tx['transaction']['message']
            # XYMの受信のみ抽出
            for mosaic in tx['transaction']['mosaics']:
                if mosaic['id'] == XYM_ID:
                    print("height:{}, {}, {}xym, [{}]".format(tx['meta']['height'], '受信' , int(mosaic['amount'])/1000000, unhexlify(message).decode('utf-8')))
    

height:275528, 受信, 1.0xym, [Hi Bob.]
